In [3]:
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
#jupyter内で画像を表示
%matplotlib inline 
#別ウィンドウで画像表示
# %matplotlib qt 
import csv
import json
from ipywidgets import interact, interactive, fixed, RadioButtons
import ipywidgets as widgets
from IPython.display import display
import pathlib
import re
import sys, math
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_columns', 100)

# モビリティとノード数の登録

In [4]:
row_no = 0
fileobj = open("no_signal_500.tcl", "r", encoding="utf_8")
# time = 14
# id_list = ['50', '100', '300', '289', '190', '898'] #ノードの位置を表示させたいIDを初期化
node_num = 500
first_x = [0] * node_num
first_y = [0] * node_num

# 最初のポジションをリストに保存
while True:
    line = fileobj.readline()
    line_split = line.split()
    if(len(line_split) == 4):
#         print(line_split)
        node_id = re.sub(r"\D", "", line_split[0])
        if(line_split[2] == 'X_'):
            x_pos = line_split[3]
            first_x[int(node_id)] = int(x_pos[:-2])
#             print('node id', node_id, 'x', first_x[int(node_id)])
        if(line_split[2] == 'Y_'):
            y_pos = line_split[3]
            first_y[int(node_id)] = int(y_pos[:-2])
#             print('node id', node_id, 'y', first_y[int(node_id)])
    if(first_x[node_num - 1] != 0):
        break

In [8]:
44
send_sigo_path = pathlib.Path('../../ns3.30/data/send_sigo/sigo-seed_20000nodenum_300.csv')
send_lsgo_path = pathlib.Path('../../ns3.30/data/send_lsgo/lsgo-seed_20000nodenum_300.csv')

44

In [9]:
send_sigo_df = pd.read_csv(send_sigo_path)
send_lsgo_df = pd.read_csv(send_lsgo_path)

In [10]:
# print('aaaaaaaaaa')
# send_lsgo_df[(send_lsgo_df['hop'] < 3) & (send_lsgo_df['time']>= 14000000) & (send_lsgo_df['time'] < 15000000)]

In [11]:
send_sigo_df['des_id'].value_counts().to_dict()

{0: 525, 1: 153}

In [12]:
send_lsgo_df['des_id'].value_counts().to_dict()

{0: 459, 1: 139}

In [13]:
dest_list =send_lsgo_df['des_id'].value_counts().index.tolist()
dest_list.sort()
len(dest_list)

2

In [14]:
def output_df(hopcount, des_id):
    # send_lsgo_df[(send_lsgo_df['hop'] < 3) & (send_lsgo_df['time']>= 14000000) & (send_lsgo_df['time'] < 15000000)]
    print('lsgo')
    display(send_lsgo_df[(send_lsgo_df['hop'] == hopcount) & (send_lsgo_df['des_id'] == des_id)])
    print('sigo')
    display(send_sigo_df[(send_sigo_df['hop'] == hopcount) & (send_sigo_df['des_id'] == des_id)])
    

In [17]:
@interact(hopcount=(1,21,1), des_id = dest_list, time=(1,17,1))
def df_run(hopcount, des_id, time):
    output_df(hopcount, des_id)
    #topologyが欲しいノードをリストで渡す
    color_list = [27,267, 153, 226, 71] #色付けたいIDを初期化
    source_id = 123
    node_num = 300
    vis_topology(time, color_list, des_id, source_id, node_num)

interactive(children=(IntSlider(value=11, description='hopcount', max=21, min=1), Dropdown(description='des_id…

# トポロジーの確認

In [16]:
def vis_topology(time, id_list, des_id, source_id, node_num):
    fileobj2 = open("test.tcl", "r", encoding="utf_8")
    row_no = 0
    x_list = []
    y_list = []
    while True:
        line = fileobj2.readline()
        line_split = line.split()
        if(len(line_split) == 8):
            split_time = line_split[2]
            split_time = split_time[:-2]
            if((int(split_time) >= time) & (int(split_time) < time + 1)):
                split_id = re.sub(r"\D", "", line_split[3])
                split_x = line_split[5]
                split_y = line_split[6]
#                 if(split_id in id_list):
                first_x[int(split_id)] = split_x
                first_y[int(split_id)] = split_y
        if line:
            row_no += 1
            if(row_no > 87575):
                break
        else:
            break
    img_area = np.zeros(shape=(1050,1050, 3), dtype=np.int16)
    plt.figure(figsize=(10, 10))
    source_color = (255,241,0)
    source_radius = 10
    des_color = (30,144,255)
    des_radius = 10
    normal_color = (0, 255, 0)
    normal_radius = 4
    target_color = (255, 0, 0)
    target_radius = 6
    for node_id in range(0, node_num):
        write_color = normal_color
        write_radius = normal_radius
        
        x = float(first_x[int(node_id)])
        y = float(first_y[int(node_id)])
        if node_id == source_id:
            write_color = source_color
            write_radius = source_radius
        if node_id in id_list:
            write_color = target_color
            write_radius = target_radius
        if node_id == des_id:
            write_color = des_color
            write_radius = des_radius
        cv2.circle(img=img_area, center=(int(x),
                           int(y)), radius=write_radius, color=write_color, thickness=-1)
    plt.grid(color='w', linestyle='dotted', linewidth=1.0)
    plt.xticks([1000,800,600,400,200, 0]) 
    plt.xlim([0, 1000])
    plt.yticks([1000,800,600,400,200, 0])
    plt.ylim([0, 1000])
    plt.imshow(img_area)